라이브러리 설치

In [ ]:
!pip install -q langchain langchain-openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


OpenAI 인증키 설정

In [ ]:
from google.colab import userdata
import os
import openai

os.environ["OPENAI_API_KEY"] = userdata.get('')


LLM Chain

1) prompt+LLM

In [ ]:
from langchain_openai import ChatOpenAI

#model
llm = ChatOpenAI(model = 'gpt-3.5-turbo-0125')

#chain 설정
content = llm.invoke("지구의 자전 주기는?")
content.content

'지구의 자전 주기는 약 24시간으로, 하루에 한 번 지구가 자전하면서 자신의 축 주위를 한 바퀴 회전합니다.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

#prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model = 'gpt-3.5-turbo-0125')
output_parser = StrOutputParser() #aimessage안의 text만 출력

#chain 설정
chain = prompt | llm | output_parser

#chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 이는 지구가 자전하는데 걸리는 시간을 의미하며, 이로 인해 하루가 낮과 밤으로 나뉘게 됩니다.'

2)Multiple Chains

In [ ]:
#chain 2개 연결
#prompt1 : 한국어단어를 영어로 번역
#prompt2 : 영어단어에 대해 설명해라
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template("explain {english_word} using oxford dictionary to me in Korean.")

llm = ChatOpenAI(model = 'gpt-3.5-turbo-0125')

chain1 = prompt1 | llm | output_parser
chain1.invoke({"korean_word":"미래"})

'Future'

In [ ]:
chain2 = (
    {"english_word" : chain1} | prompt2 | llm | output_parser
)
#chain1 >> Future
chain2.invoke({'korean_word' : "미래"}
)

'미래는 "future"라는 단어로 영어로 정의됩니다. "미래"란 어떤 일이나 사건이 발생할 때 일어날 것으로 예상되는 시간적인 범위를 가리킵니다. 일반적으로 미래는 현재 시간 이후의 시간을 의미하며, 미래에는 다양한 가능성과 전망이 포함될 수 있습니다.'

3. Prompt

1) PromptTemplate

- PromptTemplate + LLMs (단일 문장 입력 -> 단일 문장 출력)
- **문자열 프롬프트**를 위한 템플릿을 생성. Python의 문자열 포맷팅 구문을 사용.
- 내용:지시사항, 몇가지 예시, 특정 맥락 및 질문 등

In [ ]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name='홍길동', age = 30 )

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [ ]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
    prompt_template +
    PromptTemplate.from_template('\n\n아버지를 아버지라 부를 수 없습니다.') +
    "\n\n{language}로 번역해주세요."
)
combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [ ]:
combined_prompt.format(name='홍길동', age = 30, language = '영어')

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model = 'gpt-3.5-turbo-0125')
chain = combined_prompt | llm | output_parser

chain.invoke({'name':'홍길동', 'age':30, 'language':'영어'})

'Hello, my name is Hong Gildong and I am 30 years old.\n\nI cannot call my father as father.'

2)ChatPromptTemplate

- ChatPromptTemplate + ChatModels (여러 메시지 입력 -> 단일 메시지 출력)
- 채팅 메시지를 원소로 갖는 리스트 형태
- 구성 : 각 채팅 메시지는 역할(role)과 내용(content)이 짝을 이루는 형태
  - 예시 : OpenAI는 AI Assistant, Human, System등의 역할(role)로 구성

In [ ]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input = "태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [ ]:
chain = chat_prompt | llm | output_parser
chain.invoke({'user_input':'태양계에서 가장 큰 행성은 무엇인가요?'})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 139,822km로 태양계에서 가장 큰 행성이며, 질량도 가장 큽니다.'

3)Message

In [ ]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}")
    ]
)

messages = chat_prompt.format_messages(user_input = "태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [ ]:
chain = chat_prompt | llm | output_parser
chain.invoke({'user_input':'태양계에서 가장 큰 행성은 무엇인가요?'})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 142,984km로 태양계에서 가장 큰 지름을 가진 행성입니다.'

5. Model Parameter <br>
1) 모델 클래스 유형 <br>


LLM

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI()

llm.invoke("한국의 대표적인 관광지 3군데를 추천해주세요.")

'\n\n1. 경복궁 - 서울에 위치한 조선 시대 왕의 궁전으로, 아름다운 전통 한옥 건물과 화려한 궁궐을 볼 수 있으며 한국의 역사와 문화를 체험할 수 있는 곳입니다.\n\n2. 제주도 - 한국의 남쪽에 위치한 도시로, 아름다운 자연 경관과 다양한 관광 명소를 제공하고 있습니다. 특히 한라산, 우도, 성산일출봉 등이 인기 있는 관광지입니다.\n\n3. 부산 해운대해수욕장 - 부산에 위치한 해수욕장으로, 깨끗한 백사장과 아름다운 해안 라인이 인기 있는 관광 명소입니다. 해수욕뿐만 아니라 다양한 레스토'

ChatModel

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("user", "{user_input}"),
])

chain = chat_prompt | chat | output_parser

chain.invoke({'user_input':'한국의 대표적인 관광지 3군데를 추천해주세요.'})

'한국의 대표적인 관광지로는 다음 세 곳을 추천드립니다:\n\n1. 경복궁 (Gyeongbokgung Palace) - 서울에 위치한 경복궁은 조선 시대의 궁궐로, 한국의 역사와 전통을 경험할 수 있는 곳입니다. 아름다운 건물과 정원, 전통적인 복식을 입은 경비병들을 볼 수 있습니다.\n\n2. 부산 해운대해수욕장 (Haeundae Beach) - 부산의 대표적인 해변으로, 맑은 바다와 긴 백사장이 아름다운 곳입니다. 여름에는 많은 사람들이 모여 즐기는 해변으로, 다양한 레스토랑과 상점도 많이 있습니다.\n\n3. 경주 (Gyeongju) - 경주는 한국의 역사와 문화가 살아 숨쉬는 도시로, 많은 유적지와 문화유산이 있는 곳입니다. 석가탑, 첨성대, 안압지 등의 유명한 관광지뿐만 아니라, 아름다운 자연 풍경도 즐길 수 있습니다.'

Message 이용

In [ ]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI()
chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 여행 전문가입니다,"),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

chain = chat_prompt | chat | output_parser

chain.invoke({'user_input':'한국의 대표적인 관광지 3군데를 추천해주세요.'})

'한국의 대표적인 관광지로는 다음 3군데를 추천드립니다:\n\n1. 경복궁 (Gyeongbokgung Palace): 서울에 위치한 경복궁은 조선 왕조의 궁궐로, 한국의 역사와 전통을 경험할 수 있는 곳입니다. 경내에는 다양한 궁궐 건물과 정원이 있어 한국의 아름다운 건축물과 문화를 감상할 수 있습니다.\n\n2. 제주도 (Jeju Island): 한국의 대표적인 관광지인 제주도는 아름다운 자연 풍경과 풍부한 문화를 경험할 수 있는 곳입니다. 화산암으로 이루어진 풍부한 자연경관과 맑은 바다, 그리고 독특한 문화와 음식을 즐길 수 있습니다.\n\n3. 부산 해운대해수욕장 (Haeundae Beach, Busan): 부산의 대표적인 해변인 해운대해수욕장은 깨끗한 백사장과 아름다운 일몰을 감상할 수 있는 인기 관광지입니다. 여름에는 해수욕과 해변에서의 여가를 즐기는 사람들로 북적이는 곳이기도 합니다.\n\n이 세 곳은 한국을 대표하는 관광지로서 자연과 역사, 문화를 모두 경험할 수 있는 장소들입니다.'

2)모델 파라미터 설정

모델에 직접 파라미터를 전달(모델 생성 시점)


In [ ]:
from langchain_openai import ChatOpenAI

#모델 파라미터 설정
params = {
    "temperature" : 0.7, #생성된 텍스트의 다양성 조정
    "max_tokens" : 100, #생성할 최대 토큰 수
}

kwargs = {
    "frequency_penalty" : 0.5, #빈도 패널티 (이미 등장한 단어의 재등장 확률 조정)
    "presence_penalty" : 0.5, #존재 패널티 (새로운 단어의 도입을 장려)
    "stop" : ["\n"] #정지 시퀀스 설정
}

#모델 인스턴스를 생성 할 때 설정
model = ChatOpenAI(model = 'gpt-3.5-turbo-0125', **params, **kwargs)

# 모델 호출
question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input=question)

# 전체 응답 출력
print(response)

content='태양계에서 가장 큰 행성은 목성입니다. 목성은 태양계 내에서 가장 크고 질량도 가장 많이 갖춘 행성으로 알려져 있습니다.' response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 29, 'total_tokens': 91}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-eaf56841-0b02-406b-891d-e548ab6824c1-0' usage_metadata={'input_tokens': 29, 'output_tokens': 62, 'total_tokens': 91}


모델에 직접 파라미터를 전달(모델 호출 시점)

In [ ]:
params = {
    "temperature" : 0.7, #생성된 텍스트의 다양성 조정
    "max_tokens" : 10, #생성할 최대 토큰 수
}

response = model.invoke(input=question, **params)

# 전체 응답 출력
print(response.content)

태양계에서 가장 큰


모델에 추가적인 파라미터를 전달
- bind 메서드 사용


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

# # binding 이전 출력
print(before_answer)

# 모델 호출 시 추가적인 인수를 전달하기 위해 bind 메서드 사용 (응답의 최대 길이를 10 토큰으로 제한)
chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

# binding 이후 출력
print(after_answer)

content='태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 142,984km로, 태양계에서 가장 큰 행성이며 지름이 가장 큽니다.' response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 58, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4feeff09-6b4f-498a-af98-1a7a6e6345da-0' usage_metadata={'input_tokens': 58, 'output_tokens': 61, 'total_tokens': 119}
content='태양계에서 가장 큰' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 58, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None} id='run-c52dd33b-20c5-4173-9f46-2ca813731cb7-0' usage_metadata={'input_tokens': 58, 'output_tokens': 10, 'total_tokens': 68}


6. Output Parsers

1)CSV Parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

['Bibimbap', 'Kimchi', 'Bulgogi', 'Japchae', 'Tteokbokki']

2)Json Parser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

#자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name : str = Field(description="name of a cusine")
    recipe : str = Field(description="recipe of a cusine")

In [ ]:
#출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe of a cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [ ]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe of a cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [ ]:

chain = prompt | model | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'Bibimbap is a Korean mixed rice dish that is typically made with white rice, vegetables, meat, and a spicy sauce. To cook Bibimbap, start by cooking white rice according to package instructions. In a skillet, cook vegetables such as spinach, carrots, bean sprouts, and mushrooms separately with a little oil and seasoning. Cook thinly sliced beef in a hot skillet with soy sauce and garlic. In'}